# Objetivo

Identificar a melhor relação entre as variáveis explicativas e a resposta. A priori, tem-se interesse em atingir todos os clientes classificados como churn, como descrito na contextualização do README.

Nesse contexto, faria sentido pensar em métricas como maximizar o recall (taxa de verdadeiro positivo) dado que o custo de se ter um falso negativo é caro. Mas, também precisa ser levado em consideração que o gasto com um público que é falso positivo é comprometedor para a campanha.

# Pacotes

In [16]:
#!pip install mlflow
#!pip install xgboost

In [1]:
from deltalake import DeltaTable, write_deltalake
import mlflow
from mlflow.models import infer_signature
import os
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score, accuracy_score, average_precision_score, brier_score_loss

# Leitura da base v1

In [2]:
dados = DeltaTable("../1.Variaveis/tmp/dados_pp_v1").to_pandas()
dados.drop(['__index_level_0__'], axis=1, inplace=True)
dados.head()

,Customer_Age,Dependent_count,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Total_Revolving_Bal,Total_Amt_Chng_Q4_Q1,Total_Ct_Chng_Q4_Q1,vfm,pmcc,...,Income_Category_5. >= 120k,Card_Category_Gold,Card_Category_Platinum,Card_Category_Silver,Education_Level_v2_1.Uneducated,Education_Level_v2_2.High School,Education_Level_v2_3.Graduate,Education_Level_v2_4.Post-Graduate,Attrition_Flag,type
0,-0.784196,0.494019,1.403132,-1.337898,0.498943,0.963894,0.282975,-0.328225,-0.175537,-0.421450,...,0,0,0,0,0,0,1,0,0,Treino
1,0.720070,0.494019,-0.525933,0.641818,1.408428,-0.165769,-1.527806,-0.194304,-0.208685,-1.054789,...,0,0,0,0,0,0,1,0,0,Treino
2,1.346848,0.494019,-0.525933,-0.348040,0.498943,0.864865,0.894171,0.056797,-0.571459,-0.686436,...,0,0,0,0,0,0,0,1,0,Treino
3,0.218648,-0.279306,-0.525933,0.641818,-1.320028,-0.412731,0.369637,0.851953,0.252749,2.406712,...,0,0,0,0,0,0,1,0,0,Treino
4,-2.539173,-1.825958,0.117089,1.631675,1.408428,-0.858972,0.346832,-1.144306,-0.064053,-0.071911,...,0,0,0,0,1,0,0,0,0,Treino


In [3]:
dados.dtypes

Customer_Age                          float64
Dependent_count                       float64
Total_Relationship_Count              float64
Months_Inactive_12_mon                float64
Contacts_Count_12_mon                 float64
Total_Revolving_Bal                   float64
Total_Amt_Chng_Q4_Q1                  float64
Total_Ct_Chng_Q4_Q1                   float64
vfm                                   float64
pmcc                                  float64
Gender_M                                int32
Marital_Status_Married                  int32
Marital_Status_Single                   int32
Marital_Status_Unknown                  int32
Income_Category_1.< 40k                 int32
Income_Category_2. >= 40k & < 60k       int32
Income_Category_3. >= 60k & < 80k       int32
Income_Category_4. >= 80k & < 120k      int32
Income_Category_5. >= 120k              int32
Card_Category_Gold                      int32
Card_Category_Platinum                  int32
Card_Category_Silver              

## Separação das bases

Se classificássemos toda a base como "não churn" ou Attrition_Flag = 0, teríamos a métrica de acurácia em aproximadamente 83%. Utilizaremos esse valor como threshold, além de avaliar outras métricas como recall (TPR), precisão e FPR.

In [3]:
dados_treino = dados[dados.type == 'Treino'].drop(['type'], axis=1)
dados_val = dados[dados.type == 'Validacao'].drop(['type'], axis=1)
dados_teste = dados[dados.type == 'Teste'].drop(['type'], axis=1)

In [4]:
X_treino = dados_treino.drop(['Attrition_Flag'], axis=1)
y_treino = dados_treino['Attrition_Flag']

X_val = dados_val.drop(['Attrition_Flag'], axis=1)
y_val = dados_val['Attrition_Flag']

In [9]:
# A representatividade absoluta da variável target é baixa nas bases de validação e teste

dados_treino['Attrition_Flag'].value_counts(), dados_val['Attrition_Flag'].value_counts(), dados_teste['Attrition_Flag'].value_counts()

(Attrition_Flag
 0    6140
 1    1175
 Name: count, dtype: int64,
 Attrition_Flag
 0    1084
 1     208
 Name: count, dtype: int64,
 Attrition_Flag
 0    1276
 1     244
 Name: count, dtype: int64)

In [11]:
dados_treino['Attrition_Flag'].value_counts()/dados_treino.shape[0], dados_val['Attrition_Flag'].value_counts()/dados_val.shape[0], dados_teste['Attrition_Flag'].value_counts()/dados_teste.shape[0]

(Attrition_Flag
 0    0.839371
 1    0.160629
 Name: count, dtype: float64,
 Attrition_Flag
 0    0.839009
 1    0.160991
 Name: count, dtype: float64,
 Attrition_Flag
 0    0.839474
 1    0.160526
 Name: count, dtype: float64)

# Modelos baselines 

Por hora, aplicaremos alguns modelos sem a tunagem dos hiperparâmetros e avaliaremos o desempenho na base de validação.

## Cria experimento

In [6]:
experiment = mlflow.create_experiment(name = 'Modelos_Baselines',
                                      artifact_location = 'Artf_Modelos_Baselies',
                                      tags = {'Environment': 'Development', 'Version': '1.0.0'}
                                      )

In [7]:
experiment

'215021480169490868'

In [5]:
experiment = mlflow.set_experiment(experiment_id='215021480169490868')

In [6]:
experiment.experiment_id

'215021480169490868'

## Random forest

In [13]:
if __name__ == '__main__':
    
    with mlflow.start_run(run_name = 'Baseline_RF', experiment_id = experiment.experiment_id) as run:

        rf = RandomForestClassifier()
        rf.fit(X_treino, y_treino)

        # Log dos parâmetros do modelo
        mlflow.log_params(rf.get_params())

        # Log das métricas na base de TREINO
        mlflow.log_metric('AUC_PR_Treino', average_precision_score(y_treino, rf.predict_proba(X_treino)[:,1]))
        mlflow.log_metric('AUC_ROC_Treino', roc_auc_score(y_treino, rf.predict_proba(X_treino)[:,1]))
        mlflow.log_metric('BS_Treino', brier_score_loss(y_treino, rf.predict_proba(X_treino)[:,1]))

            # OBS: o ponto de corte utilizado é o 0.5 (ainda não otimizamos esse ponto)
        mlflow.log_metric('F1_Treino', f1_score(y_treino, rf.predict(X_treino)))
        mlflow.log_metric('Precision_Treino', precision_score(y_treino, rf.predict(X_treino)))
        mlflow.log_metric('Precision_Treino', recall_score(y_treino, rf.predict(X_treino)))
        mlflow.log_metric('Precision_Treino', accuracy_score(y_treino, rf.predict(X_treino)))

        # Log das métricas na base de VALIDAÇÃO
        mlflow.log_metric('AUC_PR_Val', average_precision_score(y_val, rf.predict_proba(X_val)[:,1]))
        mlflow.log_metric('AUC_ROC_Val', roc_auc_score(y_val, rf.predict_proba(X_val)[:,1]))
        mlflow.log_metric('BS_Val', brier_score_loss(y_val, rf.predict_proba(X_val)[:,1]))

                    # OBS: o ponto de corte utilizado é o 0.5 (ainda não otimizamos esse ponto)
        mlflow.log_metric('F1_Val', f1_score(y_val, rf.predict(X_val)))
        mlflow.log_metric('Precision_Val', precision_score(y_val, rf.predict(X_val)))
        mlflow.log_metric('Precision_Val', recall_score(y_val, rf.predict(X_val)))
        mlflow.log_metric('Precision_Val', accuracy_score(y_val, rf.predict(X_val)))
        
        # Log do schema das variáveis do modelo e do modelo
        signature = infer_signature(X_treino, rf.predict_proba(X_treino))
        mlflow.sklearn.log_model(rf, signature=signature, artifact_path='modelo')

/home/hugo/Documents/Git_GitHub/Estudo_Cartao_Credito/vCartao_Credito/lib/python3.12/site-packages/mlflow/types/utils.py:435: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
